# Quiz2/HW3 Hackathon

Team name: HONEYBADGERRS

Member names: Ryan Richardson, Jarret Guillow, Grace Gupta

Use this Jupyter Notebook as a template to get started. Before you start, here are a few hints that might help you

### Iterate your solution based on a simple first submission

For the quiz, try to use the numerical variables first. Fit a regression model on the numerical variables first and create a submission quickly. You can then try various other things and iterate your code based on the simple solution. Use Google to find code to solve the problems you encounter—e.g., how to select only the categorical variables in the DataFrame, how to impute missing data, etc.

### NAs (missing values) in both training set and test set

In the dataset you are working with in this competition, you will find that both the training set and test set have NAs (missing values). If the test set has missing values, your model will not be able to predict on the entire set. How do you deal with it? (Hint: one idea is to concatenate the training set and test set, fill in NAs with the means/median/mode or whatever you choose, then split them up again).

### Categorical variables

We have not discussed how to deal with categorical variables. They do not fit that easily into regression models. But how do we fully utilize them nontheless? (Hint: convert them into dummy variables, typically using one-hot encoding. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html)

### Model Evaluation

Your model will be evaluated using its Root Mean Squared Error (MSE) on the test set. (As usual, you do not have the labels on the test set, of course; so you cannot directly measure the RMSE on it.) You can calculate the MSE on a dataset using scikit-learn https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html. Then take the squared root of the MSE to get the RMSE.

In [1]:
import pandas as pd
import numpy as np
train = pd.read_csv('/Users/gracegupta/Desktop/housing_data/train.csv')

In [2]:
train.head()

,Id,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,0,20,RL,NaN,9248,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,7,2009,WD,Normal,173000
1,1,70,RM,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,10,2006,WD,Normal,157000
2,2,160,RM,24.0,1950,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,7,2008,COD,Normal,151000
3,3,20,RL,NaN,9790,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2009,WD,Normal,161500
4,4,50,RL,60.0,8064,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdWo,NaN,0,9,2008,WD,Normal,138000


In [3]:
train.shape

(2001, 81)

Get numeric data

In [4]:
train_numeric = train._get_numeric_data()

In [5]:
train_numeric.shape #there are 38 numeric columns including ID and SalePrice

(2001, 38)

Get categorical data

In [6]:
categorical_cols = [] #contains labels for all categorical columns
for col in train.columns:
    if col not in train_numeric.columns:
        categorical_cols.append(col)

In [7]:
train_categorical = train[categorical_cols]

In [8]:
train_categorical.shape #there are 43 categorical columns

(2001, 43)

# Dealing with categorical data

Look at distribution of null values

In [9]:
np.sum(train_categorical.isnull())

MS Zoning            0
Street               0
Alley             1872
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        16
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           53
Bsmt Cond           53
Bsmt Exposure       56
BsmtFin Type 1      53
BsmtFin Type 2      54
Heating              0
Heating QC           0
Central Air          0
Electrical           1
Kitchen Qual         0
Functional           0
Fireplace Qu       936
Garage Type        104
Garage Finish      106
Garage Qual        106
Garage Cond        106
Paved Drive          0
Pool QC           1989
Fence             1633
Misc Feature      1929
Sale Type            0
Sale Condition       0
dtype: int6

Drop columns with more than 1000 null values

In [10]:
train_cat_new = train_categorical.dropna(thresh=1000, axis=1)

In [11]:
np.sum(train_cat_new.isnull())

MS Zoning           0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       16
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          53
Bsmt Cond          53
Bsmt Exposure      56
BsmtFin Type 1     53
BsmtFin Type 2     54
Heating             0
Heating QC          0
Central Air         0
Electrical          1
Kitchen Qual        0
Functional          0
Fireplace Qu      936
Garage Type       104
Garage Finish     106
Garage Qual       106
Garage Cond       106
Paved Drive         0
Sale Type           0
Sale Condition      0
dtype: int64

In [12]:
train_cat_new.shape

(2001, 39)

# Using Cramer's V to identify multicollinearity among categorical variables

In [13]:
import scipy
from scipy import stats as ss

In [14]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x,y)
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

In [15]:
for col in train_cat_new.columns: #col = x
    for col2 in train_cat_new.columns: #col2 = y
        if col2 != col:
            score = cramers_v(train_cat_new[col], train_cat_new[col2])
            if score > 0.5:
                print(col, col2) 
                print(score)

MS Zoning Utilities
0.998748591675947
MS Zoning Neighborhood
0.5234232681237401
MS Zoning Heating QC
0.5126112548776743
Utilities MS Zoning
0.998748591675947


/Users/gracegupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


Neighborhood MS Zoning
0.5234232681237401
Neighborhood Exter Qual
0.5053663672565124
Exterior 1st Exterior 2nd
0.7754885964857037
Exterior 2nd Exterior 1st
0.7754885964857037
Exter Qual Neighborhood
0.5053663672565124
Exter Qual Kitchen Qual
0.5578627340476368
Heating QC MS Zoning
0.5126112548776743
Kitchen Qual Exter Qual
0.5578627340476368
Garage Qual Garage Cond
0.5237743296463221
Garage Cond Garage Qual
0.5237743296463221


# Fit model to features w/ low Cramer's V

In [16]:
train_cat_new.columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl',
       'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type', 'Exter Qual',
       'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure',
       'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC',
       'Central Air', 'Electrical', 'Kitchen Qual', 'Functional',
       'Fireplace Qu', 'Garage Type', 'Garage Finish', 'Garage Qual',
       'Garage Cond', 'Paved Drive', 'Sale Type', 'Sale Condition'],
      dtype='object')

In [17]:
col_to_exclude = ['MS Zoning', 'Exterior 2nd','Garage Cond', 'Sale Type','Heating QC','Exter Qual','Garage Type','Garage Finish',
                 'Condition 1','Condition 2','Street','Lot Shape','Land Contour','Utilities','Lot Config','Land Slope','Bldg Type',
                 'Roof Style','Roof Matl','Exterior 1st','Mas Vnr Type','Exter Cond','Foundation','Bsmt Cond','BsmtFin Type 1',
                 'BsmtFin Type 2','Central Air','Electrical','Functional','Garage Qual','Sale Condition']

In [18]:
X_train_cat = train_cat_new.drop(col_to_exclude, axis=1)

In [19]:
X_train_cat.columns

Index(['Neighborhood', 'House Style', 'Bsmt Qual', 'Bsmt Exposure', 'Heating',
       'Kitchen Qual', 'Fireplace Qu', 'Paved Drive'],
      dtype='object')

In [20]:
X_train_cat.shape

(2001, 8)

In [21]:
X_train_cat_dummies = pd.get_dummies(X_train_cat) #when creating dummies, all nulls get dropped

In [22]:
X_train_cat_dummies.shape

(2001, 64)

In [23]:
y = train_numeric["SalePrice"]

In [24]:
X_train_cat_dummies = X_train_cat_dummies.reindex(y.index) #reindex dummies

In [25]:
import statsmodels.api as sm

model_cat = sm.OLS(y, sm.add_constant(X_train_cat_dummies)).fit()

In [26]:
model_cat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.782
Model:                            OLS   Adj. R-squared:                  0.776
Method:                 Least Squares   F-statistic:                     118.1
Date:                Sun, 23 Feb 2020   Prob (F-statistic):               0.00
Time:                        18:05:31   Log-Likelihood:                -23961.
No. Observations:                2001   AIC:                         4.804e+04
Df Residuals:                    1941   BIC:                         4.838e+04
Df Model:                          59                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.925e+16   8.48e+16     -0.227      0.820   -1.86e+17    1.47e+17
Neighborhood_Blmngtn  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Blueste  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_BrDale   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_BrkSide  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_ClearCr  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_CollgCr  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Crawfor  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Edwards  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Gilbert  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Greens   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_GrnHill  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_IDOTRR   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Landmrk  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_MeadowV  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Mitchel  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_NAmes    -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_NPkVill  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_NWAmes   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_NoRidge  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_NridgHt  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_OldTown  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_SWISU    -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Sawyer   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_SawyerW  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Somerst  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_StoneBr  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Timber   -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
Neighborhood_Veenker  -8.05e+14   3.55e+15     -0.227      0.820   -7.76e+15    6.15e+15
House Style_1.5Fin   -2.034e+15   8.96e+15     -0.227      0.820   -1.96e+16    1.55e+16
House Style_1.5Unf   -2.034e+15   8.96e+15     -0.227      0.820   -1.96e+16    1.55e+16
House Style_1Story   -2.034e+15   8.96e+15     -0.227    

# Dealing with numeric data

Look at distribution of nulls

In [27]:
np.sum(train_numeric.isnull())

Id                   0
MS SubClass          0
Lot Frontage       332
Lot Area             0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Mas Vnr Area        16
BsmtFin SF 1         1
BsmtFin SF 2         1
Bsmt Unf SF          1
Total Bsmt SF        1
1st Flr SF           0
2nd Flr SF           0
Low Qual Fin SF      0
Gr Liv Area          0
Bsmt Full Bath       1
Bsmt Half Bath       1
Full Bath            0
Half Bath            0
Bedroom AbvGr        0
Kitchen AbvGr        0
TotRms AbvGrd        0
Fireplaces           0
Garage Yr Blt      106
Garage Cars          1
Garage Area          1
Wood Deck SF         0
Open Porch SF        0
Enclosed Porch       0
3Ssn Porch           0
Screen Porch         0
Pool Area            0
Misc Val             0
Mo Sold              0
Yr Sold              0
SalePrice            0
dtype: int64

In [28]:
def preprocessing_pipeline(train):
    ## Your code goes here
    
    #fill NAs with median
    train = train.fillna(train.median())

    #get rid of outliers
    for X in train:
        mu = np.mean(train[X])
        std = np.std(train[X])
        normalized_data = (train[X]-mu)/std
        indexes = normalized_data < 3
        processed_train = train.loc[indexes, :]
    
    return processed_train

Preprocess data

In [29]:
train_numeric = preprocessing_pipeline(train_numeric)

In [30]:
train_numeric.shape

(1969, 38)

Drop the "Id" column.

In [31]:
train_numeric = train_numeric.drop(columns=['Id'])

# Calculate VIFs for numeric data


In [32]:
def calculate_vif(r_squared):
    ## Your code goes here
    vif = 1/(1-r_squared)
    return vif

In [33]:
import statsmodels.api as sm

def generate_vif_dataframe(processed_train):
    ## Your code goes here
    data = []
    for X in processed_train:
        X_list = []
        X_list.append(X)
        i = processed_train.columns.get_loc(X)
        reg_list = [] #contains all other variables for regression against X
        for j in range(len(processed_train.columns)):
            if j != i :
                reg_list.append(processed_train.columns[j])
        model = sm.OLS(processed_train[X], sm.add_constant(processed_train[reg_list])).fit()
        rsq = model.rsquared
        vif = calculate_vif(rsq)
        X_list.append(vif)
        data.append(X_list)
        
    vif_dataframe = pd.DataFrame(data, columns = ['Variable name','VIF'])
                                
    return vif_dataframe

Select all features

In [34]:
numeric_features = train_numeric.loc[:, train_numeric.columns != 'SalePrice']

In [35]:
vif_dataframe = generate_vif_dataframe(numeric_features)
vif_dataframe.sort_values('VIF',ascending=False)

/Users/gracegupta/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until


,Variable name,VIF
14,Low Qual Fin SF,inf
15,Gr Liv Area,inf
12,1st Flr SF,inf
13,2nd Flr SF,inf
8,BsmtFin SF 1,1.440326e+04
10,Bsmt Unf SF,1.412280e+04
11,Total Bsmt SF,1.341112e+04
9,BsmtFin SF 2,2.005389e+03
25,Garage Cars,5.729113e+00
26,Garage Area,5.452137e+00


# Building model based on VIFs

In [36]:
columns_to_exclude = ['BsmtFin SF 1','Garage Cars','Year Built','Total Bsmt SF','TotRms AbvGrd','Garage Yr Blt','Full Bath','Low Qual Fin SF','Gr Liv Area','1st Flr SF',
                     'Low Qual Fin SF','Gr Liv Area','1st Flr SF','2nd Flr SF','Year Remod/Add','Garage Area','MS SubClass','Mo Sold','Screen Porch','Open Porch SF','Lot Area',
                     '3Ssn Porch','Misc Val']
features_updated = numeric_features.drop(columns_to_exclude, axis=1)
vifs = generate_vif_dataframe(features_updated)
vifs.sort_values('VIF',ascending=False)

,Variable name,VIF
1,Overall Qual,1.619907
5,Bsmt Unf SF,1.603250
6,Bsmt Full Bath,1.496130
9,Bedroom AbvGr,1.296856
11,Fireplaces,1.280649
3,Mas Vnr Area,1.251343
8,Half Bath,1.243503
0,Lot Frontage,1.196242
12,Wood Deck SF,1.168807
10,Kitchen AbvGr,1.155302


In [37]:
features_updated.columns

Index(['Lot Frontage', 'Overall Qual', 'Overall Cond', 'Mas Vnr Area',
       'BsmtFin SF 2', 'Bsmt Unf SF', 'Bsmt Full Bath', 'Bsmt Half Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'Fireplaces',
       'Wood Deck SF', 'Enclosed Porch', 'Pool Area', 'Yr Sold'],
      dtype='object')

In [38]:
features_updated.shape

(1969, 16)

In [39]:
y = train_numeric["SalePrice"]

In [40]:
y.shape

(1969,)

In [41]:
#features_updated = features_updated.reindex(y_test.index)

Fit model to chosen numeric features

In [42]:
model_numeric = sm.OLS(y, sm.add_constant(features_updated)).fit()

In [43]:
model_numeric.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.741
Method:                 Least Squares   F-statistic:                     353.6
Date:                Sun, 23 Feb 2020   Prob (F-statistic):               0.00
Time:                        18:05:33   Log-Likelihood:                -23405.
No. Observations:                1969   AIC:                         4.684e+04
Df Residuals:                    1952   BIC:                         4.694e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.639e+06   1.22e+06      2.159      0.031    2.42e+05    5.04e+06
Lot Frontage     395.0697     42.007      9.405      0.000     312.687     477.453
Overall Qual    3.054e+04    737.079     41.432      0.000    2.91e+04     3.2e+04
Overall Cond     264.7959    753.387      0.351      0.725   -1212.731    1742.323
Mas Vnr Area      40.8584      5.403      7.562      0.000      30.261      51.455
BsmtFin SF 2      -0.8401      4.971     -0.169      0.866     -10.589       8.909
Bsmt Unf SF        9.8089      2.272      4.318      0.000       5.353      14.264
Bsmt Full Bath   2.01e+04   1848.297     10.877      0.000    1.65e+04    2.37e+04
Bsmt Half Bath  3444.9481   3506.457      0.982      0.326   -3431.845    1.03e+04
Half Bath       8147.2356   1762.073      4.624      0.000    4691.494    1.16e+04
Bedroom AbvGr   7187.7730   1079.133      6.661      0.000    5071.399    9304.147
Kitchen AbvGr  -1.047e+04   4181.863     -2.503      0.012   -1.87e+04   -2264.515
Fireplaces      1.391e+04   1410.171      9.865      0.000    1.11e+04    1.67e+04
Wood Deck SF      51.5407      6.711      7.680      0.000      38.380      64.701
Enclosed Porch   -16.6806     13.064     -1.277      0.202     -42.301       8.940
Pool Area        -56.0240     20.458     -2.738      0.006     -96.146     -15.902
Yr Sold        -1354.5473    608.795     -2.225      0.026   -2548.503    -160.591
==============================================================================
Omnibus:                      263.043   Durbin-Watson:                   2.038
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3172.894
Skew:                          -0.042   Prob(JB):                         0.00
Kurtosis:                       9.218   Cond. No.                     3.22e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.22e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Make model from both categorical and continuous data.

In [44]:
X_train = pd.concat([X_train_cat_dummies, features_updated],axis=1)

In [45]:
X_train = X_train.dropna(axis=0)

In [46]:
X_train.shape

(1969, 80)

In [47]:
y = train_numeric["SalePrice"]

In [48]:
y.shape

(1969,)

In [49]:
model_both = sm.OLS(y, sm.add_constant(X_train)).fit()

In [50]:
model_both.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.839
Method:                 Least Squares   F-statistic:                     137.9
Date:                Sun, 23 Feb 2020   Prob (F-statistic):               0.00
Time:                        18:05:33   Log-Likelihood:                -22907.
No. Observations:                1969   AIC:                         4.597e+04
Df Residuals:                    1893   BIC:                         4.639e+04
Df Model:                          75                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 1.303e+06   5.26e+05      2.475      0.013    2.71e+05    2.34e+06
Neighborhood_Blmngtn  4.254e+04   1.99e+04      2.141      0.032    3575.031    8.15e+04
Neighborhood_Blueste  1.624e+04   2.23e+04      0.728      0.467   -2.75e+04       6e+04
Neighborhood_BrDale   3311.8966   2.02e+04      0.164      0.870   -3.63e+04     4.3e+04
Neighborhood_BrkSide  2.859e+04   1.93e+04      1.484      0.138   -9205.206    6.64e+04
Neighborhood_ClearCr  5.396e+04   1.95e+04      2.771      0.006    1.58e+04    9.21e+04
Neighborhood_CollgCr  4.982e+04    1.9e+04      2.617      0.009    1.25e+04    8.72e+04
Neighborhood_Crawfor  6.181e+04   1.91e+04      3.236      0.001    2.43e+04    9.93e+04
Neighborhood_Edwards  2.407e+04    1.9e+04      1.264      0.206   -1.33e+04    6.14e+04
Neighborhood_Gilbert   4.21e+04   1.92e+04      2.198      0.028    4532.692    7.97e+04
Neighborhood_Greens   1.888e+04   2.17e+04      0.869      0.385   -2.37e+04    6.15e+04
Neighborhood_GrnHill  1.843e+05   3.27e+04      5.633      0.000     1.2e+05    2.48e+05
Neighborhood_IDOTRR   2.067e+04   1.93e+04      1.073      0.283   -1.71e+04    5.84e+04
Neighborhood_Landmrk  2.402e+04   3.24e+04      0.740      0.459   -3.96e+04    8.77e+04
Neighborhood_MeadowV  1.031e+04   1.99e+04      0.519      0.604   -2.87e+04    4.93e+04
Neighborhood_Mitchel  3.811e+04   1.92e+04      1.987      0.047     500.887    7.57e+04
Neighborhood_NAmes    3.234e+04   1.89e+04      1.707      0.088   -4813.252    6.95e+04
Neighborhood_NPkVill  2.813e+04   2.06e+04      1.367      0.172   -1.22e+04    6.85e+04
Neighborhood_NWAmes    4.51e+04   1.91e+04      2.357      0.019    7577.295    8.26e+04
Neighborhood_NoRidge  1.031e+05   1.95e+04      5.290      0.000    6.49e+04    1.41e+05
Neighborhood_NridgHt  7.891e+04   1.92e+04      4.104      0.000    4.12e+04    1.17e+05
Neighborhood_OldTown  2.374e+04   1.92e+04      1.240      0.215   -1.38e+04    6.13e+04
Neighborhood_SWISU    2.141e+04   1.96e+04      1.090      0.276   -1.71e+04    5.99e+04
Neighborhood_Sawyer   3.101e+04   1.91e+04      1.624      0.104   -6430.073    6.85e+04
Neighborhood_SawyerW   4.15e+04   1.93e+04      2.146      0.032    3577.364    7.94e+04
Neighborhood_Somerst  6.316e+04   1.92e+04      3.296      0.001    2.56e+04    1.01e+05
Neighborhood_StoneBr  7.576e+04   1.97e+04      3.847      0.000    3.71e+04    1.14e+05
Neighborhood_Timber   6.434e+04   1.93e+04      3.332      0.001    2.65e+04    1.02e+05
Neighborhood_Veenker  7.566e+04   1.99e+04      3.803      0.000    3.66e+04    1.15e+05
House Style_1.5Fin    1.671e+05    6.6e+04      2.533      0.011    3.77e+04    2.96e+05
House Style_1.5Unf     1.58e+05    6.6e+04      2.395      0.017    2.86e+04    2.87e+05
House Style_1Story    1.667e+05   6.59e+04      2.529    

# Predict on test data

In [51]:
#Read the test set in
test = pd.read_csv('/Users/gracegupta/Desktop/housing_data/test.csv')

In [52]:
test.shape

(929, 80)

In [53]:
test_numeric = test._get_numeric_data()

In [54]:
test_numeric.shape

(929, 37)

Preprocess data

In [55]:
test_numeric = preprocessing_pipeline(test_numeric)

In [56]:
test_numeric.shape

(929, 37)

Drop the "Id" column.

In [57]:
test_numeric = test_numeric.drop(columns=['Id'])

In [58]:
test_numeric = test_numeric[features_updated.columns]

In [59]:
test_numeric.shape

(929, 16)

Get categorical test data

Use dummy variables

In [60]:
categorical_cols = [] #contains labels for all categorical columns
for col in test.columns:
    if col not in test_numeric.columns:
        categorical_cols.append(col)

In [61]:
test_categorical = test[train_cat_new.columns]

In [62]:
test_categorical.shape

(929, 39)

In [63]:
test_categorical_dummies = pd.get_dummies(test_categorical)

In [64]:
test_categorical_dummies.shape

(929, 236)

In [65]:
missing_cols = set( X_train_cat_dummies.columns ) - set( test_categorical_dummies.columns )

In [66]:
missing_cols

{'Heating_Floor', 'Kitchen Qual_Po', 'Neighborhood_Landmrk'}

In [67]:
for c in missing_cols:
    test_categorical_dummies[c] = 0

In [68]:
test_categorical_dummies = test_categorical_dummies[X_train_cat_dummies.columns]

In [69]:
test_categorical_dummies.shape

(929, 64)

In [70]:
X_train_cat_dummies.shape

(2001, 64)

In [71]:
X_test = pd.concat([test_categorical_dummies, test_numeric], axis=1)

In [72]:
X_test.shape

(929, 80)

In [73]:
pred = model_both.predict(sm.add_constant(X_test))

In [74]:
sample_submission = pd.read_csv('/Users/gracegupta/Desktop/housing_data/sample_submission (1).csv')
sample_submission.loc[:, 'SalePrice'] = pred
sample_submission.to_csv('hw3_attemp3.csv', header=True, index=False)
sample_submission.head()

,Id,SalePrice
0,0,118200.159986
1,1,229735.419149
2,2,93663.127722
3,3,161446.110280
4,4,177742.315434
